In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Install Detectron2 Dependencies

In [ ]:
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# import libraries
import numpy as np
import os, cv2, json, random
from google.colab.patches import cv2_imshow

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Import Data

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("train_dataset", {}, "/detection_dataset/train/_annotations.coco.json", "/detection_dataset/train")
register_coco_instances("test_dataset", {}, "/detection_dataset/valid/_annotations.coco.json", "/detection_dataset/valid")

# Visualize training data

In [ ]:
my_dataset_train_metadata = MetadataCatalog.get("train_dataset")
dataset_dicts = DatasetCatalog.get("train_dataset")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Detectron2 Training

In [ ]:
from detectron2.engine import DefaultTrainer

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 600
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

# Visualise training results with tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "detection_model.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.DATASETS.TEST = ("test_dataset", )
predictor = DefaultPredictor(cfg)

# Inference with Detectron2 Saved Weights



In [ ]:
import glob
from detectron2.utils.visualizer import ColorMode

In [ ]:
test_metadata = MetadataCatalog.get("test_dataset")

In [ ]:
for images in glob.glob('/detection_dataset/test/*jpg'):
  im = cv2.imread(images)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


# Evaluation

In [ ]:
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [ ]:
evaluator = COCOEvaluator("_dataset_", output_dir= "/models/output")
val_loader = build_detection_test_loader(cfg, "_dataset_")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
# Save cofig

In [ ]:
f = open('config.yml', 'w')
f.write(cfg.dump())
f.close()